In [3]:
!pip install pymysql
!pip install requests

  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached urllib3-1.25.10-py2.py3-none-any.whl (127 kB)
  Using cached certifi-2020.6.20-py2.py3-none-any.whl (156 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)


In [1]:
import json
import pymysql as pms
import requests
import base64
import logging
import zipfile as zf
import os

In [2]:
# DB에 연결
host = "ict.cor8kkyfcogd.ap-northeast-2.rds.amazonaws.com"
port = 3306
username = "admin"
database = "ict"
password = "48834883"

try:
    conn = pms.connect(host=host,user=username,password=password,db=database, 
                       port=port, use_unicode=True, charset="utf8")
    cursor=conn.cursor()
except:
    logging.error("연결 실패")
    

In [3]:
# 현재 디렉토리 파일 목록 읽어오기
path = "./"
# ipynb 체크포인트랑 이 노트북은 제외
# 124844.json부터 85795.json까지
filelist = os.listdir(path)[1:-1]
print(filelist[0],filelist[-1])

124844.json 85795.json


In [5]:
with open (filelist[0]) as file:
    json_d = json.load(file)
    print(json_d['precedent']['casename'])

손해배상(기)등



In [ ]:
# 파일명으로부터 json파일 읽어와서 sql문으로 DB에 저장
for i in filelist:
    with open(i) as json_file:
        json_data = json.load(json_file)
        query = "INSERT INTO Precedent VALUES('{0}', '{1}')".format(int(json_data['precedent']['id']), json_data['precedent']['casename'])
        cursor.execute(query)
        conn.commit()
        idx = 0
        for weight in json_data['content']['key_weight']:
            query = "INSERT INTO Content (Precedent_ID,keyIndex,keyWeight) VALUES('{0}', '{1}', '{2}')".format(int(json_data['content']['id']), idx, weight)
#             query = "INSERT INTO Content (Precedent_ID,keyIndex,keyWeight) VALUES(%d, %d, %f)"
#            print(type(int(json_data['content']['id'])))
#             val = (int(json_data['content']['id']), idx, weight)
#             print(type(val[0]))
#             cursor.execute(query, val)
            cursor.execute(query)
            idx +=1
            conn.commit()